In [1]:
import numpy as np
import argparse
from tqdm import tqdm
import time
import os, sys, inspect
os.chdir('../train')
pwdpath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
from pathlib import Path
# sys.path.insert(0, str(Path(pwdpath).parent / 'train'))

%reload_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from dataset_openroomsV2 import openrooms
from utils.utils_misc import *
from utils.utils_dataloader import make_data_loader
from utils.config import cfg

import utils.utils_config as utils_config
from utils.utils_envs import set_up_envs

parser = argparse.ArgumentParser()
# The locationi of training set
parser.add_argument('--data_root', default=None, help='path to input images')
parser.add_argument('--task_name', type=str, default='tmp', help='task name (e.g. N1: disp ref)')
parser.add_argument('--task_split', type=str, default='train', help='train, val, test', choices={"train", "val", "test"})
# Fine tune the model
parser.add_argument('--isFineTune', action='store_true', help='fine-tune the model')
parser.add_argument("--if_val", type=str2bool, nargs='?', const=True, default=True)
parser.add_argument("--if_vis", type=str2bool, nargs='?', const=True, default=True)
parser.add_argument('--epochIdFineTune', type=int, default = 0, help='the training of epoch of the loaded model')
# The training weight
parser.add_argument('--albedoWeight', type=float, default=1.5, help='the weight for the diffuse component')
parser.add_argument('--normalWeight', type=float, default=1.0, help='the weight for the diffuse component')
parser.add_argument('--roughWeight', type=float, default=0.5, help='the weight for the roughness component')
parser.add_argument('--depthWeight', type=float, default=0.5, help='the weight for depth component')   

# Cascae Level
parser.add_argument('--cascadeLevel', type=int, default=0, help='the casacade level')

# Rui
# Device
parser.add_argument("--local_rank", type=int, default=0)
# parser.add_argument("--master_port", type=str, default='8914')

# DEBUG
parser.add_argument('--debug', action='store_true', help='Debug eval')

parser.add_argument('--ifMatMapInput', action='store_true', help='using mask as additional input')
parser.add_argument('--ifDataloaderOnly', action='store_true', help='benchmark dataloading overhead')
parser.add_argument('--if_cluster', action='store_true', help='if using cluster')
parser.add_argument('--if_hdr_input_matseg', action='store_true', help='if using hdr images')
parser.add_argument('--eval_every_iter', type=int, default=2000, help='')
parser.add_argument('--save_every_iter', type=int, default=2000, help='')
parser.add_argument('--debug_every_iter', type=int, default=2000, help='')
parser.add_argument('--max_iter', type=int, default=-1, help='')
parser.add_argument('--invalid_index', type=int, default = 0, help='index for invalid aread (e.g. windows, lights)')

# Pre-training
parser.add_argument('--resume', type=str, help='resume training; can be full path (e.g. tmp/checkpoint0.pth.tar) or taskname (e.g. tmp); [to continue the current task, use: resume]', default='NoCkpt')
parser.add_argument('--reset_latest_ckpt', action='store_true', help='remove latest_checkpoint file')
parser.add_argument('--reset_scheduler', action='store_true', help='')
parser.add_argument('--reset_lr', action='store_true', help='')
# debug
parser.add_argument("--mini_val", type=str2bool, nargs='?', const=True, default=False)
# to get rid of
parser.add_argument('--test_real', action='store_true', help='')

parser.add_argument(
    "--config-file",
    default=os.path.join(pwdpath, "configs/config.yaml"),
    metavar="FILE",
    help="path to config file",
    type=str,
)
parser.add_argument(
    "params",
    help="Modify config options using the command-line",
    default=None,
    nargs=argparse.REMAINDER,
)

# The detail model setting
opt = parser.parse_args('--task_name tmp --config-file ../train/configs/config.yaml DATASET.num_workers 16 SOLVER.ims_per_batch 4 TEST.ims_per_batch 4 \
    MODEL_MATSEG.enable True MODEL_BRDF.enable True MODEL_BRDF.enable_BRDF_decoders True DATA.im_height 240 DATA.im_width 320 MODEL_BRDF.enable_list al_no_de_ro \
    MODEL_SEMSEG.config_file ../train/configs/openrooms/openrooms_pspnet50.yaml DATASET.mini True'.split())
cfg.merge_from_file(opt.config_file)
cfg = utils_config.merge_cfg_from_list(cfg, opt.params)

opt.cfg = cfg
set_up_envs(opt)
opt.cfg.freeze()

semseg_configs = utils_config.load_cfg_from_cfg_file(os.path.join(pwdpath, opt.cfg.MODEL_SEMSEG.config_file))
semseg_configs = utils_config.merge_cfg_from_list(semseg_configs, opt.params)
opt.semseg_configs = semseg_configs

opt.pwdpath = pwdpath

logger = None
opt.num_gpus = 1
opt.distributed = False

# >>>> DATASET
from utils.utils_semseg import get_transform_semseg, get_transform_matseg
transforms_train_semseg = get_transform_semseg('train', opt)
transforms_val_semseg = get_transform_semseg('val', opt)
transforms_train_matseg = get_transform_matseg('train', opt)
transforms_val_matseg = get_transform_matseg('val', opt)

brdf_dataset_train = openrooms(opt, 
    transforms_fixed = transforms_val_semseg, 
    transforms = transforms_train_semseg, 
    transforms_matseg = transforms_train_matseg,
    cascadeLevel = opt.cascadeLevel, split = 'train', logger=logger)
brdf_loader_train = make_data_loader(
    opt,
    brdf_dataset_train,
    is_train=True,
    start_iter=0,
    logger=logger,
    # collate_fn=my_collate_seq_dataset if opt.if_padding else my_collate_seq_dataset_noPadding,
)
if opt.cfg.MODEL_SEMSEG.enable:
    opt.semseg_colors = brdf_dataset_train.semseg_colors

if 'mini' in opt.cfg.DATASET.dataset_path:
    print('=====!!!!===== mini: brdf_dataset_val = brdf_dataset_train')
    brdf_dataset_val = brdf_dataset_train
    brdf_dataset_val_vis = brdf_dataset_train
else:
    brdf_dataset_val = openrooms(opt, 
        transforms_fixed = transforms_val_semseg, 
        transforms = transforms_val_semseg, 
        transforms_matseg = transforms_val_matseg,
        # cascadeLevel = opt.cascadeLevel, split = 'val', logger=logger)
        cascadeLevel = opt.cascadeLevel, split = 'val', load_first = 20 if opt.mini_val else -1, logger=logger)
brdf_loader_val = make_data_loader(
    opt,
    brdf_dataset_val,
    is_train=False,
    start_iter=0,
    logger=logger,
    # pin_memory = False, 
    # collate_fn=my_collate_seq_dataset if opt.if_padding else my_collate_seq_dataset_noPadding,
    if_distributed_override=opt.cfg.DATASET.if_val_dist and opt.distributed
)
# <<<< DATASET

In [2]:
for i, data_batch in tqdm(enumerate(brdf_loader_val)):
    print(i)


1it [00:26, 26.41s/it]

0


2it [00:26, 18.61s/it]

1
2
3
4
5


7it [00:27, 13.07s/it]

6
7
8


10it [00:29,  9.37s/it]

9
10
11
12
13
14
15


16it [00:31,  1.98s/it]


KeyboardInterrupt: 